In [1]:
import logging
from gw_scanner import GravWaveScanner
from gw_multi_process import MultiGwProcessor

NameError: name 'HTTPError' is not defined

In [ ]:
logger = logging.getLogger("quiet_logger")
logger.setLevel(logging.ERROR)

GravWaveScanner can be initialised with optional arguments *gw_name* and *rev*. By default, it loads the most recent LIGO public event, and the most recent revision. Alternatively, a fits file can be saved to the LIGO_skymaps directory, and its name specified with *gw_file*.

In [ ]:
gw = MultiGwProcessor(prob_threshold=0.99, fast_query=True, logger=logger)
# gw = GravWaveScanner(gw_file="http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2019/bn190831332/quicklook/glg_healpix_all_bn190831332.fit", prob_threshold=0.9, logger=logger)

In [ ]:
fig = gw.plot_skymap()

In [ ]:
gw.clean_cache()
gw.fill_queue()
gw.terminate()
gw.combine_cache()
gw.clean_cache()

In [ ]:
(fig, message) = gw.plot_overlap_with_observations()

In [ ]:
fig = gw.plot_ztf_observations()

Alternative methods for starting the GW scanner:

In [ ]:
# gw = GravWaveScanner(logger=logger) # Grab most recent
# gw = GravWaveScanner("S190814bv", prob_threshold=0.99, logger=logger) # Grab most recent revision of "S190814bv" up to 99%
# gw = GravWaveScanner(gw_file="https://wiki.gw-astronomy.org/pub/OpenLVEM/FermiGBM-LVC/GBM-190816-with-LV-v2.fits", prob_threshold=0.9, logger=logger) # Use manual file